# Improved Cattle Feed Model

Notebook to extract and test improved cattle feed model.

In [1]:
solution_name = 'improvedcattlefeed'

## Extract Solution

In [2]:
import os
from pathlib import Path
from tools import solution_xls_extract as sxe

excel_filepath = os.path.expanduser('~/Developer/drawdown/excel/ImprovedCattleFeed_Apr2021.xlsm')
output_dir = os.path.expanduser(f'~/Developer/drawdown/solutions/solution/{solution_name}')

Path(output_dir).mkdir(parents=True, exist_ok=True)

sxe.output_solution_python_file(outputdir=output_dir, xl_filename=excel_filepath)
print(f'Contents of {output_dir}:' + '\n\t' + '\n\t'.join(os.listdir(output_dir)))

/Users/anshul/anaconda3/envs/pd-dev/lib/python3.9/site-packages/openpyxl/reader/drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)
/Users/anshul/anaconda3/envs/pd-dev/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/anshul/anaconda3/envs/pd-dev/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/anshul/anaconda3/envs/pd-dev/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Contents of /Users/anshul/Developer/drawdown/solutions/solution/improvedcattlefeed:
	vma_data
	ca_pds_data
	.DS_Store
	ac
	ad
	tests
	tam
	__init__.py
	__pycache__


### Set PDS1, PDS2, PDS3
The code below automatically replaces code in `__init__.py` to set the values for `PDS1`, `PDS2`, and `PDS3` to the three scenario names.

In [3]:
init_filepath = os.path.join(output_dir, '__init__.py')

with open(init_filepath, 'r') as f:
    init_contents = f.read()

init_contents = init_contents.replace('PDS1 = \"NOT SET\"', 'PDS1 = \"PDS-28p2050-low7scen\"', 1)
init_contents = init_contents.replace('PDS2 = \"NOT SET\"', 'PDS2 = \"PDS-45p2050-avg7scen\"', 1)
init_contents = init_contents.replace('PDS3 = \"NOT SET\"', 'PDS3 = \"PDS-62p2050-high7scen\"', 1)

with open(init_filepath, 'w') as f:
    f.write(init_contents)

print(f'''Set values for variables in __init__.py:
\tPDS1 = \"PDS-28p2050-low7scen\"
\tPDS2 = \"PDS-45p2050-avg7scen\"
\tPDS3 = \"PDS-62p2050-high7scen\"
''')

Set values for variables in __init__.py:
	PDS1 = "PDS-28p2050-low7scen"
	PDS2 = "PDS-45p2050-avg7scen"
	PDS3 = "PDS-62p2050-high7scen"



### Bug Workaround
The code below automatically adds some python code to the `__init__.py` for the improved cattle feed model. This added code is a workaround for a bug that I am not able to fix. The workaround is needed to import and construct the improved cattle feed model, which is necessary for the remainder of the notebook to run.

In [4]:
init_filepath = os.path.join(output_dir, '__init__.py')
fix = """
    # This is a manually added VMA to get around an issue with importing this solution module: During import, function
    # _substitute_vma() in model/advanced_controls.py throws an exception because this VMA is not generated here by
    # the model extraction notebook Improved_Cattle_Feed_Extraction.ipynb. The source of that issue should be found so
    # that this potentially dangerous work-around can be removed.
    'SOLUTION Operating Cost per Functional Unit per Annum': vma.VMA(
        filename=None, use_weight=False, fixed_summary=(1e-20, 1e-20, 1e-20)),
"""

with open(init_filepath, 'r') as f:
    init_contents = f.read()

vma_dict_start = init_contents.index('VMAs = {')
num_open_brackets = 1
i = vma_dict_start + len('VMAs = {')
while num_open_brackets > 0:
    if init_contents[i] == '{':
        num_open_brackets += 1
    
    if init_contents[i] == '}':
        num_open_brackets -= 1
    
    i += 1
vma_dict_end = (i-1)

init_contents = init_contents[:vma_dict_end] + fix + init_contents[vma_dict_end:]
with open(init_filepath, 'w') as f:
    f.write(init_contents)

print(f'''
Added VMA \"SOLUTION Operating Cost per Functional Unit per Annum\" to VMAs dictionary in python file
{init_filepath} as a work around
for an Exception that otherwise occurs when using function factory() on {solution_name}.
''')


Added VMA "SOLUTION Operating Cost per Functional Unit per Annum" to VMAs dictionary in python file
/Users/anshul/Developer/drawdown/solutions/solution/improvedcattlefeed/__init__.py as a work around
for an Exception that otherwise occurs when using function factory() on improvedcattlefeed.



## Sniff Test

In [5]:
from solution import factory

my_scenario = factory.load_scenario(solution_name, "PDS1")

In [6]:
print(f'\nCO2 reduction for scenario PDS1:')
my_scenario.c2.co2_mmt_reduced()


CO2 reduction for scenario PDS1:


,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America,China,India,EU,USA
Year,,,,,,,,,,
2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2020,138.739974,41.978534,9.389152,38.612281,18.371685,30.395195,0.0,0.0,0.0,0.0
2021,161.863303,48.974957,10.954011,45.047661,21.433632,35.461061,0.0,0.0,0.0,0.0
2022,184.986632,55.971379,12.518870,51.483041,24.495580,40.526927,0.0,0.0,0.0,0.0


## Create Test Results
**This step requires the Excel application, and thus only can be run on Windows or Mac.** Create a clean temporary directory to generate the test set in. Put (a copy of) your Excel spreadsheet in that directory. Follow the instructions in tools/CREATING_EXPECTED_ZIP.md to create the CSV files in that directory.